# Toy Project

### ▪ 목표
사전 학습된 라벨링 모델을 사용하여 소비 내역 데이터를 분석하고, 그 결과를 Gradio의 챗봇 형식으로 사용자에게 제공

In [1]:
# !pip install gradio openai python-dotenv langchain langchain_community sentence-transformers tabulate

In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from typing import List
import gradio as gr
from transformers import pipeline, BertTokenizer
from kobert_transformers import get_kobert_model
import requests
import sqlite3

import os
import fitz  # PyMuPDF for PDF processing
import pickle
from datetime import datetime, timedelta  # datetime 모듈 임포트

# Langchain 관련 라이브러리
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 환경 변수 로드를 위한 라이브러리
from dotenv import load_dotenv

# Langchain 추가 라이브러리
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, BaseOutputParser
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 가상환경 확인 및 env 파일 Load 
virtual_env = os.environ.get('VIRTUAL_ENV')
if virtual_env:
    print("Virtual environment is active.")
    print("Virtual Environment Path:", virtual_env)
else:
    print("No virtual environment is active.")

# .env 파일 로드
print('.env loaded : ', load_dotenv())

# 환경 변수에서 API 키 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

# API_KEY가 제대로 로드되었는지 확인
if not API_KEY:
    raise ValueError("API_KEY가 설정되지 않았습니다. .env 파일에 OPENAI_API_KEY를 설정하세요.")

# LLM 초기화
llm = ChatOpenAI(temperature=0.1, openai_api_key=API_KEY)

No virtual environment is active.
.env loaded :  True


# 1. [DF 작업] 각 계좌내역과 이름 넣기

In [4]:
df = pd.read_csv('전처리_내역.csv')
df.head(3)

,거래일시,적요,의뢰인/수취인,내통장표시내용,출금금액,입금금액,잔액,취급점,구분
0,2024-07-01 17:15:47,체크카드,시외버스모바일(티,NaN,"12,800",0,"33,578",KB카드,
1,2024-07-01 17:15:09,체크카드,시외버스모바일(티,NaN,0,"12,800","46,378",KB카드,취소
2,2024-07-01 16:29:37,체크카드,시외버스모바일(티,NaN,"12,800",0,"33,578",KB카드,취소된거래


In [5]:
# 현재 데이터프레임의 행 수 확인
num_rows = len(df)

# '이름'과 'account_number' 컬럼 추가
df.insert(0, '이름', ['한윤지' if i < 100 else '손아현' for i in range(num_rows)])
df.insert(1, '계좌번호', ['12345' if i < 100 else '67890' for i in range(num_rows)])

display(df.head(3))
display(df.tail(3))

,이름,계좌번호,거래일시,적요,의뢰인/수취인,내통장표시내용,출금금액,입금금액,잔액,취급점,구분
0,한윤지,12345,2024-07-01 17:15:47,체크카드,시외버스모바일(티,NaN,"12,800",0,"33,578",KB카드,
1,한윤지,12345,2024-07-01 17:15:09,체크카드,시외버스모바일(티,NaN,0,"12,800","46,378",KB카드,취소
2,한윤지,12345,2024-07-01 16:29:37,체크카드,시외버스모바일(티,NaN,"12,800",0,"33,578",KB카드,취소된거래


,이름,계좌번호,거래일시,적요,의뢰인/수취인,내통장표시내용,출금금액,입금금액,잔액,취급점,구분
150,손아현,67890,2024-04-06 12:40:05,전자금융,한윤지,NaN,0,"13,000","13,061",카뱅,
151,손아현,67890,2024-04-03 12:56:19,스마트출금,한윤지,한윤지,"600,000",0,61,청주중,
152,손아현,67890,2024-04-03 12:55:36,전자금융,임진희,NaN,0,"600,000","600,061",신용협동,


In [6]:
h_df = df[df['이름']=="한윤지"]
s_df = df[df['이름']=="손아현"]

# 2. [로그인] 함수 설정

In [7]:
# 데이터베이스 연결 및 테이블 생성
def init_db():
    conn = sqlite3.connect('user_transactions.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS user_metadata (
        username TEXT,
        account_number TEXT,
        table_name TEXT,
        PRIMARY KEY (username, account_number)
    )
    ''')
    conn.commit()
    return conn

# 사용자 데이터 저장
def save_user_data(conn, username, account_number, df):
    cursor = conn.cursor()
    table_name = f"transactions_{username}_{account_number}"
    
    # 메타데이터 저장
    cursor.execute('''
    INSERT OR REPLACE INTO user_metadata (username, account_number, table_name)
    VALUES (?, ?, ?)
    ''', (username, account_number, table_name))
    
    # 트랜잭션 데이터 저장
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()

# 사용자 데이터 불러오기
def load_user_data(conn, username, account_number):
    cursor = conn.cursor()
    cursor.execute('''
    SELECT table_name FROM user_metadata
    WHERE username = ? AND account_number = ?
    ''', (username, account_number))
    result = cursor.fetchone()
    
    if result:
        table_name = result[0]
        return pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    else:
        return None

# 로그인 및 데이터 조회 함수
def login(username, account_number):
    conn = init_db()
    user_data = load_user_data(conn, username, account_number)
    if user_data is not None:
        conn.close()
        return f"로그인 성공! 환영합니다, {username}님!", gr.update(value=user_data.to_dict('records'), visible=True)
    else:
        conn.close()
        return f"{username}님의 거래 내역을 찾을 수 없습니다.", gr.update(visible=False)

In [8]:
# 데이터 저장
def save_data():
    conn = init_db()
    save_user_data(conn, "한윤지", "A001", h_df)
    save_user_data(conn, "손아현", "B002", s_df)
    
    conn.close()

# 데이터 저장
save_data()

In [9]:
#저장 데이터 출력해보기
def query_user_transactions(username, account_number, db_path='user_transactions.db'):
    # 데이터베이스 연결
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    try:
        # 사용자의 거래 내역 테이블 이름 조회
        cursor.execute('''
        SELECT table_name FROM user_metadata
        WHERE username = ? AND account_number = ?
        ''', (username, account_number))
        
        result = cursor.fetchone()
        
        if result:
            table_name = result[0]
            # 거래 내역 조회
            df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
            return df
        else:
            return None
    finally:
        conn.close()

# 사용 예시
username = "한윤지"  # 조회할 사용자 이름
account_number = "A001"  # 조회할 계좌번호

transactions = query_user_transactions(username, account_number)

if transactions is not None:
    print(f"{username}님의 거래 내역:")
    print(transactions)
else:
    print(f"{username}님의 거래 내역을 찾을 수 없습니다.")

한윤지님의 거래 내역:
     이름   계좌번호                 거래일시     적요         의뢰인/수취인  내통장표시내용    출금금액  \
0   한윤지  12345  2024-07-01 17:15:47   체크카드       시외버스모바일(티     None  12,800   
1   한윤지  12345  2024-07-01 17:15:09   체크카드       시외버스모바일(티     None       0   
2   한윤지  12345  2024-07-01 16:29:37   체크카드       시외버스모바일(티     None  12,800   
3   한윤지  12345  2024-07-01 13:45:19  스마트출금         서울세외한윤진  서울세외한윤진  80,000   
4   한윤지  12345  2024-07-01 10:22:00   기일출금  35460904034367     None  24,590   
..  ...    ...                  ...    ...             ...      ...     ...   
95  한윤지  12345  2024-05-06 12:29:38   체크카드       세븐일레븐국민대후     None  10,400   
96  한윤지  12345  2024-05-05 19:53:39   체크카드            탐앤탐스     None   6,500   
97  한윤지  12345  2024-05-05 18:10:58   체크카드           행복한약국     None   3,000   
98  한윤지  12345  2024-05-05 13:42:24   전자금융             한윤지     None       0   
99  한윤지  12345  2024-05-05 12:34:02   체크카드          바스켓테이블     None   6,500   

      입금금액       잔액   취급점     구분  
0  

In [10]:
transactions.head()

,이름,계좌번호,거래일시,적요,의뢰인/수취인,내통장표시내용,출금금액,입금금액,잔액,취급점,구분
0,한윤지,12345,2024-07-01 17:15:47,체크카드,시외버스모바일(티,None,"12,800",0,"33,578",KB카드,
1,한윤지,12345,2024-07-01 17:15:09,체크카드,시외버스모바일(티,None,0,"12,800","46,378",KB카드,취소
2,한윤지,12345,2024-07-01 16:29:37,체크카드,시외버스모바일(티,None,"12,800",0,"33,578",KB카드,취소된거래
3,한윤지,12345,2024-07-01 13:45:19,스마트출금,서울세외한윤진,서울세외한윤진,"80,000",0,"46,378",청주중,
4,한윤지,12345,2024-07-01 10:22:00,기일출금,35460904034367,None,"24,590",0,"126,378",수유동,


# 3. [함수 구현 시나리오] 자동분석

In [11]:
df = transactions.copy()

## 3-1. 카드 추천 - 근 3개월 고정

In [12]:
df['거래일시'] = pd.to_datetime(df['거래일시'], format='%Y-%m-%d')  # 날짜 형식에 맞게 수정

# 현재 날짜 계산
current_date = pd.to_datetime(datetime.now().date())

# 최근 5개월 전 날짜 계산
five_months_ago = current_date - pd.DateOffset(months=5)

# 최근 5개월 내의 데이터 필터링
recent_5_months_df = df[df['거래일시'] >= five_months_ago]
recent_5_months_df.head(3)

,이름,계좌번호,거래일시,적요,의뢰인/수취인,내통장표시내용,출금금액,입금금액,잔액,취급점,구분
0,한윤지,12345,2024-07-01 17:15:47,체크카드,시외버스모바일(티,None,"12,800",0,"33,578",KB카드,
1,한윤지,12345,2024-07-01 17:15:09,체크카드,시외버스모바일(티,None,0,"12,800","46,378",KB카드,취소
2,한윤지,12345,2024-07-01 16:29:37,체크카드,시외버스모바일(티,None,"12,800",0,"33,578",KB카드,취소된거래


In [13]:
# 카드 추천 함수 (기존 유지, 이름만 변경 - (구)get_qa_chain )
def card_recommendation():
    try:
        with open("vectordb.pkl", "rb") as f:
            vectordb = pickle.load(f)

        texts = [doc.page_content for doc in vectordb.docstore._dict.values()]
        bm25_retriever = BM25Retriever.from_texts(texts)
        bm25_retriever.k = 5

        faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, faiss_retriever],
            weights=[0.5, 0.5]
        )

        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

        prompt_template = """당신은 카드 추천 시스템입니다. PDF 문서의 내용을 기반으로 사용자의 소비 패턴과 요구에 맞는 카드를 추천해 주세요.
        주어진 컨텍스트를 사용하여 질문에 정확하고 간결하게 답변해 주세요.

        컨텍스트:
        {context}

        질문: {question}

        답변: ### 카드 추천:
        1. **추천 카드 이름**: {카드 이름}

        2. **추천 이유**: 
        - 사용자 소비 패턴 분석 결과, 이 카드는 {특정 지출 카테고리}에서 높은 포인트 적립률을 제공합니다.
        - 사용자가 자주 사용하는 {특정 상점 또는 서비스}에서 추가 할인 혜택을 받을 수 있습니다.
        - {사용자의 요구 사항 또는 선호도}에 따라, 이 카드는 {특정 혜택 예: 연회비 면제, 해외 결제 수수료 없음 등}이 있어 적합합니다.

        3. **카드 혜택 정보**:
        - **적립 혜택**: {예: 모든 구매 금액의 2% 적립, 특정 카테고리 5% 캐시백}
        - **할인 혜택**: {예: 특정 상점에서 10% 할인, 주유소 5% 할인}
        - **부가 혜택**: {예: 여행 보험, 공항 라운지 이용권, 무료 커피 쿠폰 등}
        - **연회비**: {예: 연 30,000원, 첫 해 연회비 무료 등}

        4. **추가 정보**:
        - **적용 조건**: {예: 월 최소 사용 금액 30만 원, 특정 기간 동안 한정}
        - **주의사항**: {예: 일부 혜택은 국내에서만 적용, 매월 적립 한도 제한 있음 등}

        ### 최종 추천:
        이 카드는 사용자의 소비 습관과 요구 사항을 가장 잘 충족하며, 다양한 혜택을 통해 경제적인 이점을 제공할 수 있습니다.
                
        """

        PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=ensemble_retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT}
        )
        return qa_chain
    except Exception as e:
        return f"QA 체인 설정 중 오류가 발생했습니다: {e}"

In [14]:
# # '월'만 추출 
# df['월'] = df['거래일시'].dt.to_period('M')  # 날짜에서 '월'만 추출하여 새로운 컬럼으로 추가
# card_transactions = df[df['적요'].str.contains('카드', na=False)]

In [15]:
import matplotlib.pyplot as plt  
import seaborn as sns

# "입금금액" -> "Deposit Amount"
# "출금금액" -> "Withdrawal Amount"
# "거래일시" -> "Transaction Date"

def plot_user_data(user_data):
    # 1. 데이터 정리
    ## 거래일시를 datetime 형식으로 변환
    user_data['거래일시'] = pd.to_datetime(user_data['거래일시'])
    ## 거래일시에서 월과 일을 추출하여 새로운 컬럼 추가
    user_data['거래월일'] = user_data['거래일시'].dt.strftime('%m-%d')

    # 2. 입금금액과 출금금액을 각각 정렬하여 시각화
    ## Seaborn 스타일 설정
    sns.set_style("whitegrid")  # "whitegrid" 스타일 설정
    custom_palette = ["navy", "orange"]
    sns.set_palette(custom_palette)

    ## 시각화
    # (ci=None으로 신뢰 구간 제거)
    sns.lineplot(x="거래월일", y=sorted(user_data['입금금액']), data=user_data, marker='o', label="Deposit Amount", linewidth=2, ci=None)
    sns.lineplot(x="거래월일", y=sorted(user_data['출금금액']), data=user_data, marker='o', label="Withdrawal Amount", linewidth=2, ci=None)
    
    plt.title("Visualization of Deposits and Withdrawals")
    plt.xlabel("Transaction Date (MM-DD)")
    plt.ylabel("Amount")
    plt.xticks(rotation=30)  # X축 레이블을 30도 회전하여 더 잘 보이도록 함
    plt.legend()

    plot_path = "user_transaction_plot.png"
    plt.savefig(plot_path)
    plt.close()
    return plot_path

# 4. [챗봇 고정 시나리오] 에이전트

## 4-1. [카드 추천] 함수 설정

In [16]:
# 카드 추천 함수 (기존 유지, 이름만 변경 - (구)get_qa_chain )
def card_recommendation():
    try:
        with open("vectordb.pkl", "rb") as f:
            vectordb = pickle.load(f)

        texts = [doc.page_content for doc in vectordb.docstore._dict.values()]
        bm25_retriever = BM25Retriever.from_texts(texts)
        bm25_retriever.k = 5

        faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, faiss_retriever],
            weights=[0.5, 0.5]
        )

        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

        prompt_template = """당신은 카드 추천 시스템입니다. PDF 문서의 내용을 기반으로 사용자의 소비 패턴과 요구에 맞는 카드를 추천해 주세요.
        주어진 컨텍스트를 사용하여 질문에 정확하고 간결하게 답변해 주세요.

        컨텍스트:
        {context}

        질문: {question}

        답변:"""

        PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=ensemble_retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT}
        )
        return qa_chain
    except Exception as e:
        return f"QA 체인 설정 중 오류가 발생했습니다: {e}"

## 4-2. [챗봇] 함수 설정

In [17]:
# 1. 사용자 정의 프롬프트 구성 (LLM 후처리용)
## 에이전트가 생성한 분석 결과를 사용자가 이해하기 쉬운 형태로 변환
post_processing_prompt = PromptTemplate(
    input_variables=["analysis_result"],
    template="""
    다음은 은행 고객의 AI 비서가 생성한 분석 결과입니다:
    {analysis_result}
    이 결과를 바탕으로 고객이 쉽게 이해할 수 있도록 요약하고, 추가적으로 유용할 만한 정보를 제공하세요.
    """
)

# 에이전트 생성 (데이터프레임 분석 수행)
agent = create_pandas_dataframe_agent(
    llm=llm,                           
    df=df,                             
    verbose=True,                      
    agent_type=AgentType.OPENAI_FUNCTIONS,
    output_parser=StrOutputParser(),   
    allow_dangerous_code=True )

# 2. 분석과 관련 없는 일반적인 질문에 대해 답변 하도록
main_prompt = ' 당신은 이름은 텅후루 입니다. 당신의 역할은 은행 고객의 AI비서가 생성한 분석 결과를 활용하여 대답하는 것입니다.'


# 3. 질문을 세 가지 범주(분석 관련, 카드 관련, 일반 질문)로 분류하는 프롬프트
analysis_check_prompt = PromptTemplate(
    input_variables=["input_query"],
    template="""
    아래의 질문이 다음 세 가지 범주 중 어느 것에 해당하는지 판단해 주세요:
    
    1. 분석 관련: 사용자가 지출, 수입, 예산 등에 대한 분석을 요청하는 경우.
    2. 카드 관련: 사용자가 신용카드, 혜택, 포인트 적립 등의 카드를 추천하거나 문의하는 경우.
    3. 일반 질문: 위의 두 카테고리와 관련 없는 경우.

    질문이 해당하는 범주 이름을 '분석 관련', '카드 관련', 또는 '일반 질문' 중 하나로만 대답하세요.

    질문: {input_query}
    답변:
    """
)

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'output_parser': StrOutputParser()} which are no longer supported.
  warnings.warn(


### 최종 챗봇 함수

In [18]:
# 챗봇 기능 통합 함수
## -> 병하 선배 'run_agent_continuously' 랑 합치기

def chatbot_fn(message, history):
    try:
        # LLM을 사용하여 질문이 분석 관련인지 확인
        classification_response = llm.invoke( input=analysis_check_prompt.format(input_query=message) )
        classification_text = classification_response.content.strip()


        # 1. 거래내역 분석 - LLM
        if "분석 관련" in classification_text:
            # 에이전트를 사용하여 질문에 대한 기본 분석 수행
            analysis_result = agent.run(
                name="텅후루",
                role="사용자의 소비내역을 기반으로 여러가지 질문에 답하고 분석 정보를 제공하는 것",
                input=message)
            # LLM을 사용하여 에이전트 결과에 대한 후처리 수행
            final_response = llm.invoke(
                input=post_processing_prompt.format(analysis_result=analysis_result)
            )
            final_text = final_response.content.strip()
        

        # 2. 카드 추천 - 따로 함수에서 LLM
        elif "카드 관련" in classification_text:
            # 카드 추천 관련 질문은 qa_chain을 통해 처리
            final_text = card_recommendation("qa_chain", question=message)
        

        # 3. 일반 질문 - LLM -> main_prompt
        else:
            # 분석과 관련 없는 일반 질문은 LLM이 직접 답변
            final_response = llm.invoke(
                input=main_prompt.format(input_query=message)
            )
            final_text = final_response.content.strip()

    except Exception as e:
        final_text = f"에러가 발생했습니다: {e}"

    return final_text

# 5. Gradio 인터페이스 설정

In [19]:
# '12345' 
# '67890'

In [20]:
with gr.Blocks() as demo:
    # 1. 상단에 로그인 후 / 기본값으로 모든 소비 데이터를 표시
    gr.Markdown("# 통장이 텅텅 돈이 후루룩")

    gr.Markdown("## 사용자 거래 데이터 및 로그인 시스템")
    
    with gr.Row():
        # 오른쪽에 로그인 폼
        username = gr.Textbox(label="사용자 이름", placeholder="사용자 이름을 입력하세요")
        password = gr.Textbox(label="비밀번호", type="password", placeholder="비밀번호를 입력하세요")
        login_button = gr.Button("로그인")
        output = gr.Textbox(label="결과", interactive=False)
        
        # 숨겨진 로그인 상태 (로그인 성공 여부 추적)
        login_state = gr.State(value=False)

    with gr.Tabs():
        with gr.Tab(label="데이터프레임"):
            user_data_table = gr.DataFrame(label="사용자 데이터", visible=False, interactive=False)

        with gr.Tab(label="시각화"):
            plot_output = gr.Image(label="시각화 결과", visible=False)  # 시각화 결과를 위한 이미지 출력

    # 로그인 버튼 클릭 후 화면에 변경사항이 나타나게 하기 위한 설정
    login_button.click(fn=login, inputs=[username, password], 
                                 outputs=[output, user_data_table, plot_output])


    # 2. 하단에 챗봇 
    with gr.Column():   
        # Gradio interface
        iface = gr.ChatInterface(
            chatbot_fn,
            title="텅후루 톡",
            description="안녕하세요! 챗봇 가계부 텅후룩입니다! 이름과 계좌번호를 '이름,계좌번호' 형식으로 입력해주세요.",
            theme="default",
            examples=[["고정지출을 분석해줘"], ["다음달 예상 지출 내역을 알려줘"], ['나의 지출 패턴을 분석해줘'], ["내 소비 패턴에 맞는 카드를 추천해줘"], ["포인트 적립이 높은 카드는 무엇인가요?"]],

            retry_btn="다시보내기 ↩",
            undo_btn="이전챗 삭제 ❌",
            clear_btn="전챗 삭제 💫",
        )
    # with gr.Column():  
    #     textbox=gr.Textbox(placeholder="어떤 것을 분석해드릴까요?", container=False, scale=7),

    
# Launch Gradio interface
demo.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
